In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import Packages.ClusteringHelper as ch
import numpy as np
from tqdm import tqdm
import pandas as pd
from collections import Counter

In [3]:
text, data = ch.read_aida_yago_conll(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv")
save = False
if save:
    text_file = open('text.txt', 'w')
    text_file.write(text)
    text_file.close()
data

,documents,tokens,indexes,word_indexes,mentions,entities,wikidatas,numeric_codes,alpha_codes
0,0,EU,"(0, 2)",0,,,,,
1,0,rejects,"(3, 10)",1,,,,,
2,0,German,"(11, 17)",2,German,Germany,http://en.wikipedia.org/wiki/Germany,11867,/m/0345h
3,0,call,"(18, 22)",3,,,,,
4,0,to,"(23, 25)",4,,,,,
...,...,...,...,...,...,...,...,...,...
285584,1392,younger,"(1390, 1397)",265,,,,,
285585,1392,brother,"(1398, 1405)",266,,,,,
285586,1392,",","(1406, 1407)",267,,,,,
285587,1392,Bobby,"(1408, 1413)",268,Bobby,Bobby_Charlton,http://en.wikipedia.org/wiki/Bobby_Charlton,4224,/m/01c8x


In [4]:
# Gold standard
ents_data = data[data['entities'] != '']
entities = ents_data['entities'].values
mentions = ents_data['mentions'].values
docs = ents_data['documents'].values
word_indexes = ents_data['word_indexes'].values
unique_entities = np.unique(entities)
gold_standard = Counter(entities)
gold_standard = dict(sorted(gold_standard.items(), key=lambda item: item[1], reverse=True))

#### Gensim word2vec

In [5]:
lowercase = True
if lowercase:
    text_splitted = [x.lower().split() for x in text]
else:
    text_splitted = [x.split() for x in text]

Model selection

In [12]:
# create the model
# from gensim.models.word2vec import Word2Vec
# from multiprocessing import cpu_count
# model = Word2Vec(text_splitted, min_count=0, workers=cpu_count())

# or select it
import os
from gensim.models import KeyedVectors
model_path = "D:\Sgmon\Documents\Magistrale\TESI\gensim-data"
# model = KeyedVectors.load_word2vec_format(os.path.join(model_path, 'word2vec-google-news-300', "word2vec-google-news-300.gz"),
#                                                        binary=True)

model = KeyedVectors.load_word2vec_format(os.path.join(model_path, 'glove-wiki-gigaword-300', "glove-wiki-gigaword-300.txt"))

# model = KeyedVectors.load_word2vec_format(os.path.join(model_path, 'fasttext-wiki-news-subwords-300', "fasttext-wiki-news-subwords-300.gz"),
#                                                        binary=False)

In [13]:
leven_cluster = np.loadtxt("../aida-yago2-dataset/db_cluster_levestein_0.txt", dtype=np.int32)
lev_cluster_dict = {}
windows = 100
doc_windows = 20
for i, x in enumerate(leven_cluster):
    try:
        lev_cluster_dict[x].append(
            (mentions[i], entities[i], docs[i], word_indexes[i],
             ch.get_context(text_splitted[docs[i]], word_indexes[i], windows=windows)))
    except:
        lev_cluster_dict[x] = [
            (mentions[i], entities[i], docs[i], word_indexes[i],
             ch.get_context(text_splitted[docs[i]], word_indexes[i], windows=windows))]
# """se uso un cluster diverso da minimo 0 allora scommento"""
# del lev_cluster_dict[-1]

In [14]:
contexts_vectorized = ch.calculate_context_vector(model, lev_cluster_dict[0], text_splitted, doc_window=30,
                                                      doc_context_alpha=0.3, word_context_aplha=0.7)

In [18]:
from sklearn.cluster import AgglomerativeClustering

clusterizator = AgglomerativeClustering(n_clusters=None, affinity='cosine', distance_threshold=.37, linkage="average")
clusters = clusterizator.fit_predict(contexts_vectorized)
clusters

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [19]:
[(lev_cluster_dict[0][i][1], x) for i,x in enumerate(clusters)]

[('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 1),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany_Davis_Cup_team', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 5),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('German_language', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('German_language', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
 ('Germany', 0),
